In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error


In [3]:
import os
from pathlib import Path
os.chdir("D:/Utkarsh/UTKARSH/ML/Projects/HOSPITAL_ADMISSIONS")
os.getcwd()

'D:\\Utkarsh\\UTKARSH\\ML\\Projects\\HOSPITAL_ADMISSIONS'

In [4]:
df_train = pd.read_csv('dataset/train.csv')
df_test = pd.read_csv('dataset/test.csv')
df_train

,Date,Hospital_ID,Department,Temperature,Precipitation,Air_Quality_Index,Flu_Activity,Public_Holiday,Weekend,Special_Events,Staffing_Level,Admissions
0,2018-01-01,H001,Emergency,32.5,0.1,88,Low,1,0,0,77.0,63
1,2018-01-01,H001,Surgery,32.5,0.1,88,High,1,0,0,69.7,23
2,2018-01-01,H001,Pediatrics,32.5,0.1,88,Low,1,0,0,69.1,12
3,2018-01-01,H001,Internal Medicine,32.5,0.1,88,High,1,0,0,63.4,23
4,2018-01-01,H001,Obstetrics,32.5,0.1,88,High,1,0,0,69.9,11
...,...,...,...,...,...,...,...,...,...,...,...,...
45645,2022-12-31,H005,Emergency,31.0,0.1,47,High,0,1,0,79.7,66
45646,2022-12-31,H005,Surgery,31.0,0.1,47,High,0,1,0,66.3,16
45647,2022-12-31,H005,Pediatrics,31.0,0.1,47,Moderate,0,1,0,65.9,14
45648,2022-12-31,H005,Internal Medicine,31.0,0.1,47,High,0,1,0,69.4,24


#### Preprocessing and Feature Engineering

In [5]:
train_df = df_train.copy()

train_df['Date'] = pd.to_datetime(train_df['Date'])

train_df = train_df.sort_values(
    ['Hospital_ID', 'Department', 'Date']
)

train_df

,Date,Hospital_ID,Department,Temperature,Precipitation,Air_Quality_Index,Flu_Activity,Public_Holiday,Weekend,Special_Events,Staffing_Level,Admissions
0,2018-01-01,H001,Emergency,32.5,0.1,88,Low,1,0,0,77.0,63
25,2018-01-02,H001,Emergency,46.8,0.1,90,Low,0,0,0,94.9,61
50,2018-01-03,H001,Emergency,34.0,0.2,72,High,0,0,0,92.9,73
75,2018-01-04,H001,Emergency,30.0,0.1,100,High,0,0,0,99.3,85
100,2018-01-05,H001,Emergency,31.3,0.0,80,High,0,0,0,91.4,70
...,...,...,...,...,...,...,...,...,...,...,...,...
45546,2022-12-27,H005,Surgery,35.8,0.1,103,High,0,0,0,94.3,47
45571,2022-12-28,H005,Surgery,22.2,0.1,57,High,0,0,0,91.5,35
45596,2022-12-29,H005,Surgery,27.7,0.0,108,High,0,0,0,95.2,38
45621,2022-12-30,H005,Surgery,32.2,0.1,93,Moderate,0,0,0,91.0,26


In [6]:
#date processing
train_df['dayofweek'] = train_df['Date'].dt.dayofweek
train_df['is_weekend'] = train_df['dayofweek'].isin([5,6]).astype(int)
train_df['weekofyear'] = train_df['Date'].dt.isocalendar().week.astype(int)
train_df['month'] = train_df['Date'].dt.month
train_df['quarter'] = train_df['Date'].dt.quarter
train_df

,Date,Hospital_ID,Department,Temperature,Precipitation,Air_Quality_Index,Flu_Activity,Public_Holiday,Weekend,Special_Events,Staffing_Level,Admissions,dayofweek,is_weekend,weekofyear,month,quarter
0,2018-01-01,H001,Emergency,32.5,0.1,88,Low,1,0,0,77.0,63,0,0,1,1,1
25,2018-01-02,H001,Emergency,46.8,0.1,90,Low,0,0,0,94.9,61,1,0,1,1,1
50,2018-01-03,H001,Emergency,34.0,0.2,72,High,0,0,0,92.9,73,2,0,1,1,1
75,2018-01-04,H001,Emergency,30.0,0.1,100,High,0,0,0,99.3,85,3,0,1,1,1
100,2018-01-05,H001,Emergency,31.3,0.0,80,High,0,0,0,91.4,70,4,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45546,2022-12-27,H005,Surgery,35.8,0.1,103,High,0,0,0,94.3,47,1,0,52,12,4
45571,2022-12-28,H005,Surgery,22.2,0.1,57,High,0,0,0,91.5,35,2,0,52,12,4
45596,2022-12-29,H005,Surgery,27.7,0.0,108,High,0,0,0,95.2,38,3,0,52,12,4
45621,2022-12-30,H005,Surgery,32.2,0.1,93,Moderate,0,0,0,91.0,26,4,0,52,12,4


In [7]:
#categorical identifiers
train_df["Hospital_ID"] = train_df["Hospital_ID"].astype("category")
train_df["Department"] = train_df["Department"].astype("category")

In [8]:
#other featurers
train_df['temperature_7d_mean'] = train_df.groupby(
    ['Hospital_ID', 'Department']
)['Temperature'].shift(1).rolling(7).mean()

train_df['is_rain'] = (train_df['Precipitation'] > 0).astype('int') #boolean for precipitation present or not
train_df['Precip_log'] = np.log1p(train_df['Precipitation']) #log transforming the outliers

train_df['AQI_7d_mean'] = train_df.groupby(
    ['Hospital_ID', 'Department']
)['Air_Quality_Index'].shift(1).rolling(7).mean()

flu_map = {'Low': 0, 'Moderate' : 1, 'High': 2}
train_df['flu_activity'] = train_df['Flu_Activity'].map(flu_map)

train_df['staffing_7d_mean'] = train_df.groupby(
    ['Hospital_ID', 'Department']
)['Staffing_Level'].shift(1).rolling(7).mean()

#keeping public holiday, weekend, and special events as is
train_df

C:\Users\lenovo\AppData\Local\Temp\ipykernel_15672\2118129026.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  train_df['temperature_7d_mean'] = train_df.groupby(
C:\Users\lenovo\AppData\Local\Temp\ipykernel_15672\2118129026.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  train_df['AQI_7d_mean'] = train_df.groupby(
C:\Users\lenovo\AppData\Local\Temp\ipykernel_15672\2118129026.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence t

,Date,Hospital_ID,Department,Temperature,Precipitation,Air_Quality_Index,Flu_Activity,Public_Holiday,Weekend,Special_Events,...,is_weekend,weekofyear,month,quarter,temperature_7d_mean,is_rain,Precip_log,AQI_7d_mean,flu_activity,staffing_7d_mean
0,2018-01-01,H001,Emergency,32.5,0.1,88,Low,1,0,0,...,0,1,1,1,NaN,1,0.095310,NaN,0,NaN
25,2018-01-02,H001,Emergency,46.8,0.1,90,Low,0,0,0,...,0,1,1,1,NaN,1,0.095310,NaN,0,NaN
50,2018-01-03,H001,Emergency,34.0,0.2,72,High,0,0,0,...,0,1,1,1,NaN,1,0.182322,NaN,2,NaN
75,2018-01-04,H001,Emergency,30.0,0.1,100,High,0,0,0,...,0,1,1,1,NaN,1,0.095310,NaN,2,NaN
100,2018-01-05,H001,Emergency,31.3,0.0,80,High,0,0,0,...,0,1,1,1,NaN,0,0.000000,NaN,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45546,2022-12-27,H005,Surgery,35.8,0.1,103,High,0,0,0,...,0,52,12,4,28.814286,1,0.095310,88.571429,2,82.128571
45571,2022-12-28,H005,Surgery,22.2,0.1,57,High,0,0,0,...,0,52,12,4,29.885714,1,0.095310,92.142857,2,82.042857
45596,2022-12-29,H005,Surgery,27.7,0.0,108,High,0,0,0,...,0,52,12,4,28.314286,0,0.000000,85.000000,2,82.671429
45621,2022-12-30,H005,Surgery,32.2,0.1,93,Moderate,0,0,0,...,0,52,12,4,27.814286,1,0.095310,83.714286,1,82.142857


In [9]:
#time based series features
#lag features

for lag in [1, 7, 14]:
    train_df[f'admissions_lag_{lag}'] = train_df.groupby(
        ['Hospital_ID', 'Department']
    )['Admissions'].shift(lag)

#rolling statistics

train_df['adm_roll_7'] = train_df.groupby(
    ['Hospital_ID', 'Department']
)['Admissions'].shift(1).rolling(7).mean()

train_df['adm_roll_14'] = train_df.groupby(
    ['Hospital_ID', 'Department']
)['Admissions'].shift(1).rolling(14).mean()

train_df


C:\Users\lenovo\AppData\Local\Temp\ipykernel_15672\3728907556.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  train_df[f'admissions_lag_{lag}'] = train_df.groupby(
C:\Users\lenovo\AppData\Local\Temp\ipykernel_15672\3728907556.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  train_df[f'admissions_lag_{lag}'] = train_df.groupby(
C:\Users\lenovo\AppData\Local\Temp\ipykernel_15672\3728907556.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default an

,Date,Hospital_ID,Department,Temperature,Precipitation,Air_Quality_Index,Flu_Activity,Public_Holiday,Weekend,Special_Events,...,is_rain,Precip_log,AQI_7d_mean,flu_activity,staffing_7d_mean,admissions_lag_1,admissions_lag_7,admissions_lag_14,adm_roll_7,adm_roll_14
0,2018-01-01,H001,Emergency,32.5,0.1,88,Low,1,0,0,...,1,0.095310,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
25,2018-01-02,H001,Emergency,46.8,0.1,90,Low,0,0,0,...,1,0.095310,NaN,0,NaN,63.0,NaN,NaN,NaN,NaN
50,2018-01-03,H001,Emergency,34.0,0.2,72,High,0,0,0,...,1,0.182322,NaN,2,NaN,61.0,NaN,NaN,NaN,NaN
75,2018-01-04,H001,Emergency,30.0,0.1,100,High,0,0,0,...,1,0.095310,NaN,2,NaN,73.0,NaN,NaN,NaN,NaN
100,2018-01-05,H001,Emergency,31.3,0.0,80,High,0,0,0,...,0,0.000000,NaN,2,NaN,85.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45546,2022-12-27,H005,Surgery,35.8,0.1,103,High,0,0,0,...,1,0.095310,88.571429,2,82.128571,32.0,33.0,28.0,27.428571,28.285714
45571,2022-12-28,H005,Surgery,22.2,0.1,57,High,0,0,0,...,1,0.095310,92.142857,2,82.042857,47.0,30.0,27.0,29.428571,29.642857
45596,2022-12-29,H005,Surgery,27.7,0.0,108,High,0,0,0,...,0,0.000000,85.000000,2,82.671429,35.0,33.0,36.0,30.142857,30.214286
45621,2022-12-30,H005,Surgery,32.2,0.1,93,Moderate,0,0,0,...,1,0.095310,83.714286,1,82.142857,38.0,41.0,45.0,30.857143,30.357143


#### Time-series validation split

In [10]:
#splitting date wise for train validation split
df_model = train_df.dropna().copy()

split_date = df_model["Date"].quantile(0.8)

train_idx = df_model["Date"] <= split_date
val_idx   = df_model["Date"] > split_date


X_tr = df_model.loc[train_idx].drop(columns=["Admissions", "Date", "Flu_Activity"])
y_tr = df_model.loc[train_idx, "Admissions"]
X_val = df_model.loc[val_idx].drop(columns=["Admissions", "Date", "Flu_Activity"])
y_val = df_model.loc[val_idx, "Admissions"]



In [11]:
cat_cols = ["Hospital_ID", "Department"]

for col in cat_cols:
    X_tr[col] = X_tr[col].astype("category").cat.codes
    X_val[col] = X_val[col].astype("category").cat.codes


### Model training and hyper parameter tuning

#### using random forest model for training and evaluating the mean absolute percentage error metric

In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error


rf = RandomForestRegressor(
    n_estimators=300,
    max_depth=15,
    min_samples_leaf=10,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_tr, y_tr)


y_val_pred = rf.predict(X_val)
y_val_pred = np.clip(y_val_pred, 1, None)

mape = mean_absolute_percentage_error(y_val, y_val_pred)
print('Validation MAPE: ',mape)

Validation MAPE:  0.10430103964719972


In [13]:
feat_imp = pd.Series(
    rf.feature_importances_,
    index=X_tr.columns
).sort_values(ascending=False)

feat_imp.head(10)

adm_roll_14          0.738794
admissions_lag_7     0.137431
Staffing_Level       0.028195
flu_activity         0.025855
admissions_lag_14    0.025519
Department           0.009546
Special_Events       0.006038
adm_roll_7           0.004274
Precip_log           0.004156
Precipitation        0.003779
dtype: float64

In [14]:
val_df = X_val.copy()
val_df["y_true"] = y_val.values
val_df["y_pred"] = y_val_pred

dept_mape = (
    val_df
    .groupby("Department")
    .apply(lambda x: mean_absolute_percentage_error(x["y_true"], x["y_pred"]))
)

dept_mape


C:\Users\lenovo\AppData\Local\Temp\ipykernel_15672\1846884487.py:8: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: mean_absolute_percentage_error(x["y_true"], x["y_pred"]))


Department
0    0.099841
1    0.104728
2    0.104825
3    0.107718
4    0.104393
dtype: float64

#### Training on entire train dataset 

In [15]:
# combine train + validation
X_full = pd.concat([X_tr, X_val])
y_full = pd.concat([y_tr, y_val])

# retrain final model
rf_final = RandomForestRegressor(
    n_estimators=300,
    max_depth=15,
    min_samples_leaf=10,
    random_state=42,
    n_jobs=-1
)

rf_final.fit(X_full, y_full)


,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",300
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",15
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",10
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",1.0
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples

### Preparing test set for predictions by applying same preprocessing/feature engineering as train set.
#### In order to ensure the correct lag features are being added, the lag entries from train set will be used for test set.

In [16]:
df_all = pd.concat([df_train, df_test], axis=0, ignore_index=True)
df_all

,Date,Hospital_ID,Department,Temperature,Precipitation,Air_Quality_Index,Flu_Activity,Public_Holiday,Weekend,Special_Events,Staffing_Level,Admissions
0,2018-01-01,H001,Emergency,32.5,0.1,88,Low,1,0,0,77.0,63.0
1,2018-01-01,H001,Surgery,32.5,0.1,88,High,1,0,0,69.7,23.0
2,2018-01-01,H001,Pediatrics,32.5,0.1,88,Low,1,0,0,69.1,12.0
3,2018-01-01,H001,Internal Medicine,32.5,0.1,88,High,1,0,0,63.4,23.0
4,2018-01-01,H001,Obstetrics,32.5,0.1,88,High,1,0,0,69.9,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...
54770,2023-12-31,H005,Emergency,32.7,0.5,73,Low,0,1,0,75.2,NaN
54771,2023-12-31,H005,Surgery,32.7,0.5,73,Low,0,1,0,61.5,NaN
54772,2023-12-31,H005,Pediatrics,32.7,0.5,73,Moderate,0,1,0,69.6,NaN
54773,2023-12-31,H005,Internal Medicine,32.7,0.5,73,High,0,1,0,67.9,NaN


In [17]:
df_all = df_all.sort_values(
    ['Hospital_ID', 'Department', 'Date']
)

df_all['Date'] = pd.to_datetime(df_all['Date'])

In [18]:
df_all['dayofweek'] = df_all['Date'].dt.dayofweek
df_all['is_weekend'] = df_all['dayofweek'].isin([5,6]).astype(int)
df_all['weekofyear'] = df_all['Date'].dt.isocalendar().week.astype(int)
df_all['month'] = df_all['Date'].dt.month
df_all['quarter'] = df_all['Date'].dt.quarter

df_all["Hospital_ID"] = df_all["Hospital_ID"].astype("category")
df_all["Department"] = df_all["Department"].astype("category")

df_all['temperature_7d_mean'] = df_all.groupby(
    ['Hospital_ID', 'Department']
)['Temperature'].shift(1).rolling(7).mean()

df_all['is_rain'] = (df_all['Precipitation'] > 0).astype('int') #boolean for precipitation present or not
df_all['Precip_log'] = np.log1p(df_all['Precipitation']) #log transforming the outliers

df_all['AQI_7d_mean'] = df_all.groupby(
    ['Hospital_ID', 'Department']
)['Air_Quality_Index'].shift(1).rolling(7).mean()

flu_map = {'Low': 0, 'Moderate' : 1, 'High': 2}
df_all['flu_activity'] = df_all['Flu_Activity'].map(flu_map)

df_all['staffing_7d_mean'] = df_all.groupby(
    ['Hospital_ID', 'Department']
)['Staffing_Level'].shift(1).rolling(7).mean()

for lag in [1, 7, 14]:
    df_all[f'admissions_lag_{lag}'] = df_all.groupby(
        ['Hospital_ID', 'Department']
    )['Admissions'].shift(lag)

#rolling statistics

df_all['adm_roll_7'] = df_all.groupby(
    ['Hospital_ID', 'Department']
)['Admissions'].shift(1).rolling(7).mean()

df_all['adm_roll_14'] = df_all.groupby(
    ['Hospital_ID', 'Department']
)['Admissions'].shift(1).rolling(14).mean()


C:\Users\lenovo\AppData\Local\Temp\ipykernel_15672\1468399581.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_all['temperature_7d_mean'] = df_all.groupby(
C:\Users\lenovo\AppData\Local\Temp\ipykernel_15672\1468399581.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_all['AQI_7d_mean'] = df_all.groupby(
C:\Users\lenovo\AppData\Local\Temp\ipykernel_15672\1468399581.py:24: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this wa

In [19]:
df_train_feat = df_all[df_all["Admissions"].notna()].copy()
df_test_feat  = df_all[df_all["Admissions"].isna()].copy()

In [20]:
df_test_feat = df_test_feat.drop(columns=['Flu_Activity'])
df_test_feat

,Date,Hospital_ID,Department,Temperature,Precipitation,Air_Quality_Index,Public_Holiday,Weekend,Special_Events,Staffing_Level,...,is_rain,Precip_log,AQI_7d_mean,flu_activity,staffing_7d_mean,admissions_lag_1,admissions_lag_7,admissions_lag_14,adm_roll_7,adm_roll_14
45650,2023-01-01,H001,Emergency,25.9,0.0,67,1,1,1,50.0,...,0,0.000000,78.714286,1,87.585714,87.0,90.0,64.0,85.428571,82.571429
45675,2023-01-02,H001,Emergency,19.7,0.2,67,0,0,0,97.5,...,1,0.182322,79.857143,2,85.185714,NaN,89.0,85.0,NaN,NaN
45700,2023-01-03,H001,Emergency,33.5,0.0,112,0,0,0,90.9,...,0,0.000000,77.428571,1,85.971429,NaN,100.0,71.0,NaN,NaN
45725,2023-01-04,H001,Emergency,29.9,0.1,88,0,0,0,87.9,...,1,0.095310,78.714286,2,85.257143,NaN,70.0,84.0,NaN,NaN
45750,2023-01-05,H001,Emergency,18.9,0.0,77,0,0,0,92.4,...,0,0.000000,83.142857,2,83.528571,NaN,80.0,71.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54671,2023-12-27,H005,Surgery,36.8,0.0,101,0,0,0,96.1,...,0,0.000000,73.285714,2,80.828571,NaN,NaN,NaN,NaN,NaN
54696,2023-12-28,H005,Surgery,35.2,0.2,98,0,0,0,100.0,...,1,0.182322,75.000000,0,81.414286,NaN,NaN,NaN,NaN,NaN
54721,2023-12-29,H005,Surgery,26.8,0.0,79,0,0,0,89.5,...,0,0.000000,79.571429,1,82.185714,NaN,NaN,NaN,NaN,NaN
54746,2023-12-30,H005,Surgery,35.0,0.4,52,0,1,0,61.2,...,1,0.336472,80.428571,1,82.785714,NaN,NaN,NaN,NaN,NaN


In [21]:
cat_cols = ["Hospital_ID", "Department"]

for col in cat_cols:
    cats = pd.Categorical(df_train_feat[col]).categories
    df_test_feat[col] = pd.Categorical(
        df_test_feat[col], categories=cats
    ).codes

In [22]:
X_test = df_test_feat[X_full.columns]

In [23]:
X_test.isna().sum().sort_values(ascending=False).head()

adm_roll_14          9100
adm_roll_7           9100
admissions_lag_1     9100
admissions_lag_7     8950
admissions_lag_14    8775
dtype: int64

In [24]:
X_test = X_test.fillna(method="ffill")
X_test = X_test.fillna(method="bfill")

C:\Users\lenovo\AppData\Local\Temp\ipykernel_15672\2806807586.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_test = X_test.fillna(method="ffill")
C:\Users\lenovo\AppData\Local\Temp\ipykernel_15672\2806807586.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_test = X_test.fillna(method="bfill")


In [25]:
y_test_pred = rf_final.predict(X_test)
y_test_pred = np.clip(y_test_pred, 1, None)

In [ ]:
# submission = df_all.loc[df_all["Admissions"].isna(), 
#                          ["Date", "Hospital_ID", "Department"]].copy()

# submission["Admissions"] = np.round(y_test_pred)

# submission.to_csv("submission.csv", index=False)

In [27]:
import lightgbm as lgb


lgbm = lgb.LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.03,
    num_leaves=31,
    max_depth=-1,
    min_data_in_leaf=20,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    force_col_wise='true'
)

lgbm.fit(
    X_tr, y_tr,
    eval_set=[(X_val, y_val)],
    eval_metric="mape"
)

y_val_pred = lgbm.predict(X_val)
y_val_pred = np.clip(y_val_pred, 1, None)

mape = mean_absolute_percentage_error(y_val, y_val_pred)
print("Validation MAPE:", mape)


[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Info] Total Bins 2613
[LightGBM] [Info] Number of data points in the train set: 36250, number of used features: 25
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Info] Start training from score 32.110841
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
Validation MAPE: 0.0937867079341912


### Saving the model and feature columns

In [28]:
import joblib
from pathlib import Path

Path("artifacts").mkdir(exist_ok=True)

joblib.dump(lgbm, "artifacts/model.joblib")
joblib.dump(list(X_full.columns), "artifacts/feature_columns.pkl")


['artifacts/feature_columns.pkl']

In [31]:
Path("artifacts").mkdir(exist_ok=True)

cat_maps = {
    "Hospital_ID": list(df_train_feat["Hospital_ID"].cat.categories),
    "Department": list(df_train_feat["Department"].cat.categories)
}

joblib.dump(cat_maps, "artifacts/category_maps.pkl")


['artifacts/category_maps.pkl']